In [2]:
import json
import csv
from pathlib import Path
import re


COLUMNS = ["KKS код", "Описание", "Ед. изм.", "Используемые сигналы", "Код"]


def iter_elements(elements):
    """elements может быть dict (как в описании) или list — обработаем оба варианта."""
    if isinstance(elements, dict):
        return elements.values()
    if isinstance(elements, list):
        return elements
    return []


def unique_preserve_order(items):
    seen = set()
    out = []
    for x in items:
        if not x:
            continue
        if x not in seen:
            seen.add(x)
            out.append(x)
    return out


def aggregate(folder: str):
    rows = []
    for path in sorted(Path(folder).glob("*.json")):
        try:
            data = json.loads(path.read_text(encoding="utf-8"))
        except Exception as e:
            # битые/не-JSON файлы пропускаем
            print(f"Skip {path.name}: {e}")
            continue

        project = data.get("project") or {}
        if project.get("type") != "parameter":
            continue

        signals = []
        for el in iter_elements(data.get("elements") or {}):
            if not isinstance(el, dict):
                continue
            if el.get("type") != "input-signal":
                continue
            props = el.get("props") or {}
            name = props.get("name")
            if isinstance(name, str):
                name = name.strip()
            elif name is not None:
                name = str(name)
            else:
                name = ""
            signals.append(name)

        signals_str = "; ".join(unique_preserve_order(signals))

        rows.append({
            "KKS код": project.get("code", ""),
            "Описание": project.get("description", ""),
            "Ед. изм.": project.get("dimension", ""),
            "Используемые сигналы": signals_str,
            "Код": data.get("code", ""),
        })

    return rows

def process_code_column(df):
    """
    Обрабатывает столбец 'Код':
    1. Заменяет AND->&&, OR->||, NOT->! (только как отдельные операторы)
    2. Заменяет § на _
    3. Для сигналов, начинающихся с цифры, добавляет 'P' в начало
    """
    
def process_code_column(df):
    """
    Обрабатывает столбец 'Код':
    1. Заменяет AND->&&, OR->||, NOT->! (только отдельные слова)
    2. Для сигналов, начинающихся с цифры, добавляет 'P' в начало
    3. Заменяет § на _
    """
    
    def process_row(row):
        code = row['Код']
        signals = row['Используемые сигналы']
        
        if pd.isna(code) or not isinstance(code, str) or code == "":
            return code
            
        # 1. Заменяем логические операторы как отдельные слова
        code = re.sub(r'\bAND\b', '&&', code)
        code = re.sub(r'\bOR\b', '||', code)
        code = re.sub(r'\bNOT\b', '!', code)
        
        # 2. Добавляем 'P' к сигналам, начинающимся с цифры
        if pd.notna(signals) and isinstance(signals, str) and signals != "":
            signal_list = [s.strip() for s in signals.split(';') if s.strip()]
            
            for signal in signal_list:
                if signal and signal[0].isdigit():
                    # ПРОСТО добавляем 'P' перед каждой найденной последовательностью
                    # Не анализируем контекст - просто заменяем все вхождения
                    code = code.replace(signal, 'P' + signal)
        
        # 3. Заменяем § на _
        code = code.replace('§', '_')
        
        return code
    
    df['Код'] = df.apply(process_row, axis=1)
    return df

def save_to_xlsx_with_formatting(df, out_path="result.xlsx"):
    # Сохраняем в XLSX
    df.to_excel(out_path, index=False, engine="openpyxl")
    
    from openpyxl import load_workbook
    from openpyxl.styles import Alignment

    wb = load_workbook(out_path)
    ws = wb.active

    # Включаем перенос текста для нужных столбцов
    wrap_cols = {
        "Описание",
        "Используемые сигналы",
        "Код",
    }

    # Найдём номера столбцов по заголовкам
    header = [cell.value for cell in next(ws.iter_rows(min_row=1, max_row=1))]
    col_idx = {name: i+1 for i, name in enumerate(header)}

    for col_name in wrap_cols:
        idx = col_idx.get(col_name)
        if not idx:
            continue
        for cell in ws.iter_cols(min_col=idx, max_col=idx, min_row=2):
            for c in cell:
                c.alignment = Alignment(wrap_text=True, vertical="top")

    # Немного подправим ширины столбцов (на вкус)
    widths = {
        "КKS код": 18,
        "Описание": 50,
        "Ед. изм.": 12,
        "Используемые сигналы": 30,
        "Код": 60,
    }
    for name, w in widths.items():
        idx = col_idx.get(name)
        if idx:
            ws.column_dimensions[ws.cell(row=1, column=idx).column_letter].width = w

    wb.save(out_path)



rows = aggregate('projects')
df = pd.DataFrame.from_records(rows).reindex(columns=COLUMNS)
df = process_code_column(df)
save_to_xlsx_with_formatting(df, 'Сигналы.xlsx')

In [9]:
import pandas as pd
from pathlib import Path
import json
from docx import Document
from docx.shared import Pt

RULE_COLUMNS = ["KKS код", "Используемые сигналы", "Код", "Описание", "Методические указания"]

def aggregate_rules(folder: str):
    rows = []
    for path in sorted(Path(folder).glob("*.json")):
        try:
            data = json.loads(path.read_text(encoding="utf-8"))
        except Exception as e:
            print(f"Skip {path.name}: {e}")
            continue

        project = data.get("project") or {}
        if project.get("type") != "rule":
            continue

        # Сигналы — точно так же, как в параметрах
        signals = []
        for el in iter_elements(data.get("elements") or {}):
            if not isinstance(el, dict):
                continue
            if el.get("type") != "input-signal":
                continue
            props = el.get("props") or {}
            name = props.get("name")
            if isinstance(name, str):
                name = name.strip()
            elif name is not None:
                name = str(name)
            else:
                name = ""
            signals.append(name)
        signals_str = "; ".join(unique_preserve_order(signals))

        rows.append({
            "KKS код": project.get("code", ""),
            "Используемые сигналы": signals_str,
            "Код": data.get("code", ""),                # берем так же, как в параметрах
            "Описание": project.get("possibleCause", ""),
            "Методические указания": project.get("guidelines", ""),
        })
    return rows

def save_rules_to_docx(df, out_path="rules.docx"):
    doc = Document()
    labels = [
        ("KKS код:", "KKS код"),
        ("Используемые сигналы:", "Используемые сигналы"),
        ("Код:", "Код"),
        ("Описание:", "Описание"),
        ("Методические указания:", "Методические указания"),
    ]
    for i, row in df.iterrows():
        for label, col in labels:
            p = doc.add_paragraph()
            run = p.add_run(label + " ")
            run.bold = True
            run.font.size = Pt(12)
            p.add_run("" if pd.isna(row[col]) else str(row[col]))
        if i != len(df) - 1:
            doc.add_page_break()
    doc.save(out_path)

# Агрегация, обработка кода и сохранение
rows_rule = aggregate_rules('projects')  # замените на вашу папку
df_rule = pd.DataFrame.from_records(rows_rule).reindex(columns=RULE_COLUMNS)
df_rule = process_code_column(df_rule)   # функция из предыдущей ячейки
save_rules_to_docx(df_rule, "Правила.docx")
print(f"OK: {len(df_rule)} объектов -> rules.docx")

OK: 10 объектов -> rules.docx
